# WiFi positioning
This Notebook intends to demonstrate how to prepare and request a position from a WiFi fingerprint using the openpositioning RESTFull API.

In [ ]:
import requests
import json

## Extracting Wi-Fi Scan Data and Sending to an API

This demonstrates how to extract Wi-Fi scan data from the protobuf file generated and send it to the Positioning API endpoint to make a POST request for processing.

###  Import Libraries and Load Protobuf Data

First, we need to import necessary libraries and load our protobuf data from a file.

In [ ]:
import traj_pb2 as Traj 

# Specify the path to the protobuf file containing the trajectory data
file_path = '919.pkt'

# Create an instance of the Trajectory class
trajectory = Traj.Trajectory()
with open(file_path, 'rb') as file:
    trajectory.ParseFromString(file.read())

Show the content of the file

In [ ]:
print(str(trajectory)[:1000])

### Extract Wi-Fi Scan Data

Next, we'll extract the first set of Wi-Fi scan data from our protobuf object and prepare it for the API request.

In [ ]:
# Accessing the first set of Wi-Fi scan data
first_wifi_scan = trajectory.wifi_data[0].mac_scans

# Iterating through each scan to extract MAC addresses and RSSI values
wf_dict = {}
for idx, scan in enumerate(first_wifi_scan):
    prop_key = str(scan.mac)
    wf_dict[prop_key] = scan.rssi
    
json_wifi_features = json.dumps({"wf": wf_dict})

Displaying the WiFi data

In [ ]:
print(json_wifi_features)

First Wi-Fi Scan: `trajectory.wifi_data[0].mac_scans` specifically accesses the mac_scans of the first item in `trajectory.wifi_data`. This means we're only working with the Wi-Fi scan data collected at the first timestamp. To access more, you simply change the index and access the required data.

### Send Data to API
Finally, we'll send the prepared Wi-Fi scan data to an API endpoint.

In [ ]:
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
}

response = requests.post('https://openpositioning.org/api/position/fine', headers=headers,data=json_wifi_features) 

if response.status_code == 200:
    print("Success:", response.text)
elif response.status_code == 422:
    print("Validation Error:", response.json())
else:
    print("Response Code:", response.status_code, response.text)

Request each FP of the trajectory:

In [ ]:
positions = {}
for wifi_index, wifi_data in enumerate(trajectory.wifi_data):
    wifi_scan = wifi_data.mac_scans
    # Iterating through each scan to extract MAC addresses and RSSI values
    wf_dict = {str(scan.mac): scan.rssi for scan in wifi_scan}
        
    json_wifi_features = json.dumps({"wf": wf_dict})
    
    response = requests.post('https://openpositioning.org/api/position/fine', headers=headers,data=json_wifi_features) 

    if response.status_code == 200:
        print("Success:", response.text)
        positions[wifi_index] = json.loads(response.text)
    elif response.status_code == 422:
        print("Validation Error:", response.json())
    else:
        print("Response Code:", response.status_code, response.text)